In [1]:
import os
import requests

user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]

headers = {"Content-Type":"application/json","Accept":"application/json"}

def get_catalog_item_collection():
    url = 'https://dev199703.service-now.com/api/now/table/sc_cat_item?sysparm_query=sc_catalogsLIKEe0d08b13c3330100c8b837659bba8fb4%5Eactive%3Dtrue'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        data = {'result':{'sys_id': "", 'sys_name': "",'short_description':"",'description':""}, 'Status': response.status_code, 'Headers': response.headers, 'Error Response':response.json()}
    data = response.json()
    return data

KeyError: 'SERVICENOW_USERNAME'

In [19]:
get_catalog_item_collection()

{'result': [{'entitlement_script': '',
   'use_sc_layout': 'true',
   'show_variable_help_on_load': 'false',
   'no_order_now': 'false',
   'sc_ic_version': '',
   'delivery_time': '1970-01-03 00:00:00',
   'sys_updated_on': '2022-11-21 04:46:33',
   'type': 'item',
   'published_ref': '',
   'no_search': 'false',
   'sys_updated_by': 'ryan.hale',
   'price': '0',
   'sys_created_on': '2014-01-30 16:04:26',
   'recurring_frequency': '',
   'sys_name': 'Request Developer Project Equipment',
   'model': '',
   'state': '',
   'no_cart': 'false',
   'sys_created_by': 'admin',
   'group': '',
   'hide_sp': 'false',
   'order': '0',
   'start_closed': 'false',
   'image': '',
   'no_quantity': 'false',
   'workflow': '',
   'delivery_plan': {'link': 'https://dev199703.service-now.com/api/now/table/sc_cat_item_delivery_plan/523da512c611228900811a37c97c2014',
    'value': '523da512c611228900811a37c97c2014'},
   'active': 'true',
   'checked_out': '',
   'custom_cart': '',
   'no_cart_v2': 'fa

In [3]:
import requests
import os
from langchain.docstore.document import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 
from langchain_chroma import Chroma
from langchain.schema import HumanMessage
import json
from bs4 import BeautifulSoup


user = 'admin'
pwd = 'wr^zd%PWRC12'

headers = {"Content-Type":"application/json","Accept":"application/json"}

embedding_function = AzureOpenAIEmbeddings(model="Text-embedding")

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [ ]:
def jsontoplaintext(jsonobj):
    

In [4]:
def get_catalog_item():
    url = 'https://dev199703.service-now.com/api/now/table/u_software'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    print(data)

In [ ]:
def loadJSONFile(data):
    docs=[]
    for catalog_item in data['result']:
        metadata = {"sys_id":catalog_item['sys_id'], "sys_name":catalog_item['sys_name']}
        content = "Catalog Name: "+catalog_item['sys_name'] \
            +",\nShort Description: "+catalog_item['short_description'] \
            +",\nDescription: "+convert_html_to_text(catalog_item['description'])
        docs.append(Document(page_content=content, metadata=metadata))
    return docs 



def get_similar_catalog_item(query):
    
    
    main_docs = loadJSONFile(data=get_catalog_item())
    db = Chroma.from_documents(main_docs, embedding_function)
    answer = db.similarity_search_with_relevance_scores(query, k=4)
    # print(answer)
    detailedList = []
    response = {}
    for document in answer:
        similarity_score = document[1]
        if(similarity_score > -1):
            value = document[0]
            answerlist = {}
            answerlist['content'] = value.page_content
            answerlist['sys_id'] = value.metadata['sys_id']
            answerlist['sys_name'] = value.metadata['sys_name']
            detailedList.append(answerlist)
   
    response = {'result': detailedList, 'response_detail':'similar_catalog_items'}
    return response

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.environ.get("SECRET_AZURE_OPENAI_API_KEY"))

None


In [118]:
import requests
import os

from dotenv import load_dotenv
load_dotenv()

base_url = os.environ["SERVICENOW_BASEURL"]
user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]
headers = {"Content-Type":"application/json","Accept":"application/json"}


def get_specific_catalog_item(sys_id):

    url = f'{base_url}/sn_sc/servicecatalog/items/{sys_id}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    return data

def get_all_variables_List(catalog_item_variable):
    def extract_variables(variables):
        variable_List = []
        for variable_info in variables:
            if variable_info['type'] == 24:
                continue
            if 'children' in variable_info:
                variable_List.extend(extract_variables(variable_info['children']))
            else:
                variable_List.append(variable_info)
        return variable_List
    
    return extract_variables(catalog_item_variable)

catalog_item = get_specific_catalog_item('50572ffac3405a1068d8b132b4013177')

In [119]:
catalog_item

{'result': {'short_description': 'Request for Software Services',
  'kb_article': '',
  'icon': 'images/service_catalog/generic_small.gifx',
  'description': '<div>\r\n<ul><li>Please select Business Function/Request Type to display delivery time</li></ul>\r\n</div>',
  'availability': 'on_desktop',
  'mandatory_attachment': False,
  'request_method': '',
  'type': 'catalog_item',
  'visible_standalone': True,
  'sys_class_name': 'sc_cat_item',
  'sys_id': '50572ffac3405a1068d8b132b4013177',
  'content_type': '',
  'order': 0,
  'make_item_non_conversational': False,
  'owner': '6816f79cc0a8016401c5a33be04be441',
  'show_price': False,
  'show_quantity': True,
  'picture': '',
  'url': '',
  'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4',
    'active': True,
    'title': 'Service Catalog'}],
  'name': 'Software Services',
  'show_wishlist': True,
  'category': {'sys_id': '01d72b3ec3405a1068d8b132b40131ac',
   'title': 'Personal Catalogs'},
  'turn_off_nowassist_conversation'

In [120]:
catalog_variables = get_all_variables_List(catalog_item['result']['variables'])

def fetch_ui_actions(action_field, catalog_variables):
    ui_action_variable_list = []
    for action_variables in action_field:
        action_variable_id = action_variables['name'].split(':')[1] 
        for variable in catalog_variables:
            if(variable['id'] == action_variable_id):
                ui_action_variable_list.append({'name': variable['name'], 'mandatory': action_variables['mandatory']})
    return ui_action_variable_list


def fetch_ui_policy(catalog_item, catalog_variables):
    final_ui_condition_action = []
    for catalog_ui_policy in catalog_item['result']['ui_policy']:
        for condition_field in catalog_ui_policy['conditions']:
            action_field = catalog_ui_policy['actions']
            ui_action_variable_list = fetch_ui_actions(action_field, catalog_variables)
            variable_id = condition_field['field'].split(':')[1]
            for variable in catalog_variables:
                if variable['id'] == variable_id:
                    ui_condition = {'condition_variable_name':variable['name'],'condition_variable_value': condition_field['value'], 'condition_operation': condition_field['oper'], 'ui_actions': ui_action_variable_list}
                    final_ui_condition_action.append(ui_condition)
    return final_ui_condition_action



In [121]:
prepared_api_request = {'existing_variables': {'business_service': 'effae772c3805a1068d8b132b40131dd',
  'software_name': '7c29b2afc3441a5068d8b132b4013164',
  'requested_for': '6816f79cc0a8016401c5a33be04be441',
  'Location': '163c49b037d0200044e0bfc8bcbe5dcc',
  'project_name': 'a581ab703710200044e0bfc8bcbe5de8'},
 'missing_variables': ['contact_number', 'select_your_host_name']}

existing_variables = prepared_api_request['existing_variables']
missing_variables = prepared_api_request['missing_variables']

for ui_policy in fetch_ui_policy(catalog_item, catalog_variables):
    if(ui_policy["condition_variable_name"] in existing_variables):
        if(ui_policy['condition_operation'] == '='):
            if(existing_variables[ui_policy["condition_variable_name"]] == ui_policy['condition_variable_value']):
                for action in ui_policy['ui_actions']:
                    if(action['mandatory'] == 'true'):
                        if(action['name'] in existing_variables):
                            continue
                        elif (action['name'] not in missing_variables):
                            missing_variables.append(action['name'])
                    print(action)

    else:
        print('else')

print(prepared_api_request)

{'name': 'request_type', 'mandatory': 'true'}
{'name': 'add_software', 'mandatory': 'ignore'}
{'name': 'duration', 'mandatory': 'true'}
{'existing_variables': {'business_service': 'effae772c3805a1068d8b132b40131dd', 'software_name': '7c29b2afc3441a5068d8b132b4013164', 'requested_for': '6816f79cc0a8016401c5a33be04be441', 'Location': '163c49b037d0200044e0bfc8bcbe5dcc', 'project_name': 'a581ab703710200044e0bfc8bcbe5de8'}, 'missing_variables': ['contact_number', 'select_your_host_name', 'request_type', 'duration']}


In [122]:


catalog_variables

[{'active': True,
  'label': 'Requested For',
  'dynamic_value_field': '',
  'type': 31,
  'mandatory': True,
  'displayvalue': 'System Administrator',
  'friendly_type': 'requested_for',
  'display_type': 'Requested For',
  'reference': 'sys_user',
  'render_label': True,
  'ref_qualifier': 'EQ',
  'read_only': False,
  'name': 'requested_for',
  'attributes': 'edge_encryption_enabled=true',
  'id': '9228637ec3405a1068d8b132b4013151',
  'value': 'javascript:gs.getUserID();',
  'dynamic_value_dot_walk_path': '',
  'help_text': '',
  'max_length': 0,
  'order': 1},
 {'active': True,
  'label': 'Contact Number',
  'dynamic_value_field': '',
  'type': 6,
  'mandatory': True,
  'displayvalue': '',
  'friendly_type': 'single_line_text',
  'display_type': 'Single Line Text',
  'render_label': True,
  'read_only': False,
  'name': 'contact_number',
  'attributes': 'edge_encryption_enabled=true',
  'id': 'cfa82f7ec3405a1068d8b132b4013157',
  'value': '',
  'dynamic_value_dot_walk_path': '',
  

In [123]:
import requests
import os

from dotenv import load_dotenv
load_dotenv()

base_url = os.environ["SERVICENOW_BASEURL"]
user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]
headers = {"Content-Type":"application/json","Accept":"application/json"}


def get_specific_catalog_item(sys_id):

    url = f'{base_url}/sn_sc/servicecatalog/items/{sys_id}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    return data

def get_all_variables_List(catalog_item_variable):
    def extract_variables(variables):
        variable_List = []
        for variable_info in variables:
            if variable_info['type'] == 24:
                continue
            if 'children' in variable_info:
                variable_List.extend(extract_variables(variable_info['children']))
            else:
                variable_List.append(variable_info)
        return variable_List
    
    return extract_variables(catalog_item_variable)

catalog_item = get_specific_catalog_item('50572ffac3405a1068d8b132b4013177')

In [124]:
import requests
import os
from langchain.docstore.document import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma

In [125]:
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [126]:
def request_api_templates(data):

    template = """{data}
    
    using this json create a template to sent api request to catalog item add cart based on the ui policy
    if ui policy has 2 scenarios based on the conditons then i need 2 api request template.

    if i have a 1 or more scenarios based on the ui_policy conditons then the api templates should be in the List and the list should be in the json the key of the result should be 'result'. 
    And no premable or explaination.

    """
    
    prompt_template = PromptTemplate(template=template, input_variables=["data"])
    chain = prompt_template | model | JsonOutputParser()
    response = chain.invoke({"data": data})
    return response

In [127]:
request_api_templates(catalog_item)

{'result': [{'requested_for': 'javascript:gs.getUserID();',
   'contact_number': '',
   'Location': '',
   'project_name': '',
   'business_service': 'effae772c3805a1068d8b132b40131dd',
   'software_name': '',
   'software_name1': '',
   'add_software': False,
   'request_type': '',
   'duration': '',
   'select_your_host_name': ''},
  {'requested_for': 'javascript:gs.getUserID();',
   'contact_number': '',
   'Location': '',
   'project_name': '',
   'business_service': '141b6772c3805a1068d8b132b40131bb',
   'software_name': '',
   'software_name1': '',
   'add_software': False,
   'request_type': '',
   'duration': '',
   'select_your_host_name': ''}]}